In [1]:
import requests
import json
import numpy as np

In [2]:
import json
import requests
from collections import defaultdict

In [3]:
from lxml import html
import pandas as pd

In [4]:
df =pd.read_csv("altmetricIDS.csv", header=None)

In [5]:
df.columns = ['Altmetrics_Id']
df.head()

,Altmetrics_Id
0,702709
1,403270
2,2394296
3,632123
4,411425


In [6]:
df["youtube_links"] = ""
df.head()

,Altmetrics_Id,youtube_links
0,702709,
1,403270,
2,2394296,
3,632123,
4,411425,


In [9]:
ids = np.array(df['Altmetrics_Id'])
ids[5255]

1969481

In [10]:
def add_video(alt_id):
    try:
        response = requests.get("https://api.altmetric.com/v1/id/" + str(alt_id))
        if 'video' in dict(json.loads(response.content)):
            return dict(json.loads(response.content))['video']
    except:
        return ''

In [121]:
param = {"key" : ""}

In [48]:
url_content = "https://www.googleapis.com/youtube/v3/videos?part=snippet&key=&id="

In [122]:
url = "https://www.googleapis.com/youtube/v3/videos?part=snippet,statistics"

In [123]:
from bs4 import BeautifulSoup
for alt_id in ids[5255:5260]:
    mylist = []
    alt_req = requests.get("https://api.altmetric.com/v1/id/" + str(alt_id))
    alt_rs= json.loads(alt_req.text)
    mylist.append(alt_rs)
    y_links = []
    html = requests.get('https://www.altmetric.com/details/' + str(alt_id) + '/video').text
    bs = BeautifulSoup(html)
    possible_links = bs.find_all('a')
    for link in possible_links:
        if link.has_attr('href') and 'youtube' in link.attrs['href']:
            y_links.append(link.attrs['href'])
    for link in y_links:
        param["id"] = link[32:]
        response_statistics = requests.get(url, param)
        json_statistics = json.loads(response_statistics.text)
        mylist.append(json_statistics)
    file = open(str(alt_id) + ".txt", "w+")
    json.dump(mylist, file)
    file.close()

In [98]:
with open("1969481.txt") as json_file:
    data = json.load(json_file)
data[0]["items"]

[{'kind': 'youtube#video',
  'etag': '"Bdx4f4ps3xCOOo1WZ91nTLkRZ_c/PP9t3Vze2v4L1KjNy2cfahjjjP4"',
  'id': 'V24_PZnkqY0',
  'snippet': {'publishedAt': '2014-10-04T04:25:54.000Z',
   'channelId': 'UCYidxsD_Gp62-Vz9LlrAxew',
   'title': 'The concept of MAYA   The illusion',
   'description': 'Simulations back up theory that Universe is a hologram\nhttp://www.nature.com/news/simulations-back-up-theory-that-universe-is-a-hologram-1.14328\n\nWhat Lord krishna given to Arjuna 5000 years ago ( and given to Vivasvan eons ago ) is beeing verifyed by the scientific community . That the universe is in fact a Holographic Projection . Science took so many years to reach this ground breaking theory that will shatter many of our world view , while the information is already available to us through our scriptures ...\n\n" Om purnamadah purnamidam purnaat purnamudachyate,purnasya purnamadaya purnamevaavashishyate"\n\n" That (consciousness) is full (perfect) ; this (the manifest universe of matter; of na

In [34]:
df[90:100]

,Altmetrics_Id,youtube_links,cited_by_fbwalls_count,cited_by_gplus_count,cited_by_msm_count,cited_by_posts_count,cited_by_tweeters_count,cited_by_videos_count,cited_by_wikipedia_count,cited_by_accounts_count,title,doi
90,1139197,https://www.youtube.com/watch?v=0DsiP9Iefbw,114.0,8.0,0.0,689.0,197.0,2.0,2.0,325.0,"Ehlers-Danlos Syndrome, Hypermobility Type: An...",10.5402/2012/751768
91,1487498,https://www.youtube.com/watch?v=efuTMPXrT6w,6.0,1.0,1.0,79.0,19.0,20.0,1.0,49.0,A saturated fatty acid–rich diet induces an ob...,10.3945/ajcn.2009.27792
92,29271525,https://www.youtube.com/watch?v=EUR3FCqh4PQ,19.0,6.0,1.0,95.0,4.0,40.0,10.0,80.0,Aggregation of blood platelets by adrenaline a...,0
93,7086992,https://www.youtube.com/watch?v=FlQbUeJIgp8,1.0,4.0,0.0,67.0,14.0,3.0,0.0,22.0,Worlds Beyond the Poles,0
94,130816,https://www.youtube.com/watch?v=AxuyU6X834A,68.0,259.0,28.0,831.0,187.0,27.0,3.0,599.0,The importance of the ratio of omega-6/omega-3...,10.1016/s0753-3322(02)00253-6
95,1391907,https://www.youtube.com/watch?v=iEeFC9Jg-ZA,7.0,0.0,22.0,158.0,23.0,38.0,3.0,99.0,Normal caffeine consumption: influence on ther...,0
96,102093,https://www.youtube.com/watch?v=pUMGvzEZzQo,152.0,44.0,192.0,2999.0,1807.0,43.0,22.0,2400.0,"RETRACTED: Ileal-lymphoid-nodular hyperplasia,...",10.1016/s0140-6736(97)11096-0
97,177895,https://www.youtube.com/watch?v=hNs2hy0jjM4,26.0,1.0,2.0,243.0,140.0,38.0,0.0,211.0,Effect of two different weight-loss rates on b...,0
98,23649493,https://www.youtube.com/watch?v=4XvvgqVUKqk,0.0,0.0,0.0,47.0,3.0,2.0,0.0,5.0,The C# Programmer’s Study Guide (MCSD),10.1007/978-1-4842-2860-9
99,1529096,https://www.youtube.com/watch?v=vQ9eJUkaBRs,127.0,9.0,49.0,821.0,444.0,27.0,2.0,678.0,Vegetarian Dietary Patterns and Mortality in A...,10.1001/jamainternmed.2013.6473


In [42]:
for i in range(len(ids[:100])):
    video_id = df.loc[i, "youtube_links"][32:]
    link = url_statistics + video_id
    #print(link)
    response_statistics = requests.get(link)
    if (response_statistics.status_code == 200):
        json_statistics = json.loads(response_statistics.text)
        for j in ['viewCount', 'likeCount', 'dislikeCount', 'commentCount']:
            try:
                df.loc[i, j] = json_statistics['items'][0]["statistics"][j]
            except KeyError:
                df.loc[i, j] = 0
            except IndexError:
                df.loc[i, j] = 0
df.head()

,Altmetrics_Id,youtube_links,cited_by_fbwalls_count,cited_by_gplus_count,cited_by_msm_count,cited_by_posts_count,cited_by_tweeters_count,cited_by_videos_count,cited_by_wikipedia_count,cited_by_accounts_count,title,doi,viewCount,likeCount,dislikeCount,commentCount
0,702709,https://www.youtube.com/watch?v=GSqdhz--yR8,31.0,16.0,1.0,934.0,11.0,13.0,2.0,74.0,HLA genes in Macedonians and the sub-Saharan o...,10.1034/j.1399-0039.2001.057002118.x,3506,94,9,20
1,403270,https://www.youtube.com/watch?v=2iqRF6mWj-A,9.0,2.0,1.0,698.0,97.0,58.0,2.0,173.0,Optimal low-density lipoprotein is 50 to 70 mg/dl,10.1016/j.jacc.2004.03.046,29609,988,292,1012
2,2394296,https://www.youtube.com/watch?v=k0HKvl5dYV0,115.0,5.0,37.0,1213.0,377.0,54.0,2.0,600.0,"Beyond Meatless, the Health Effects of Vegan D...",10.3390/nu6062131,44094,5153,259,1357
3,632123,https://www.youtube.com/watch?v=GpfLHrtdcS4,62.0,83.0,1.0,1036.0,169.0,48.0,0.0,365.0,Twenty questions on atherosclerosis,0,65,5,0,1
4,411425,https://www.youtube.com/watch?v=2iqRF6mWj-A,23.0,8.0,8.0,896.0,333.0,27.0,1.0,410.0,Low-carbohydrate diets and all-cause and cause...,10.1059/0003-4819-153-5-201009070-00003,29609,988,292,1012


In [45]:
y_link = url_statistics +"GSqdhz--yR8"
rs_st = requests.get(y_link)
rs_st_d = json.loads(rs_st.text)
rs_st_d

{'kind': 'youtube#videoListResponse',
 'etag': '"Bdx4f4ps3xCOOo1WZ91nTLkRZ_c/Y3aouuiaqUMW7WefSHvXculge6I"',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 1},
 'items': [{'kind': 'youtube#video',
   'etag': '"Bdx4f4ps3xCOOo1WZ91nTLkRZ_c/g3GZB4XiF2wtWRZPplKjYwvXKj8"',
   'id': 'GSqdhz--yR8',
   'statistics': {'viewCount': '3506',
    'likeCount': '94',
    'dislikeCount': '9',
    'favoriteCount': '0',
    'commentCount': '20'}}]}

In [49]:
y_content = url_content + "GSqdhz--yR8"
rs_ct = requests.get(y_link)
rs_ct_d = json.loads(rs_st.text)
rs_ct_d

{'kind': 'youtube#videoListResponse',
 'etag': '"Bdx4f4ps3xCOOo1WZ91nTLkRZ_c/Y3aouuiaqUMW7WefSHvXculge6I"',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 1},
 'items': [{'kind': 'youtube#video',
   'etag': '"Bdx4f4ps3xCOOo1WZ91nTLkRZ_c/g3GZB4XiF2wtWRZPplKjYwvXKj8"',
   'id': 'GSqdhz--yR8',
   'statistics': {'viewCount': '3506',
    'likeCount': '94',
    'dislikeCount': '9',
    'favoriteCount': '0',
    'commentCount': '20'}}]}

In [22]:
social_media_columns = ['cited_by_fbwalls_count', 'cited_by_gplus_count', 'cited_by_msm_count', 'cited_by_posts_count', 
                        'cited_by_tweeters_count', 'cited_by_videos_count', 'cited_by_wikipedia_count', 
                        'cited_by_accounts_count',]

In [23]:
for i in range(len(ids[:100])):
    rs = requests.get("https://api.altmetric.com/v1/id/" + str(ids[i]))
    if (rs.status_code == 200):
        rs_json = json.loads(rs.text)
        for col in social_media_columns:
            try:
                df.loc[i, col] = rs_json[col]
            except KeyError:
                df.loc[i, col] = 0

In [ ]:
df[60:80]

In [25]:
cols = ['title', 'doi']

In [26]:
for i in range(len(ids[:100])):
    rs = requests.get("https://api.altmetric.com/v1/id/" + str(ids[i]))
    if (rs.status_code == 200):
        rs_json = json.loads(rs.text)
        for col in cols:
            try:
                df.loc[i, col] = rs_json[col]
            except KeyError:
                df.loc[i, col] = 0

In [27]:
df.head()

,Altmetrics_Id,youtube_links,cited_by_fbwalls_count,cited_by_gplus_count,cited_by_msm_count,cited_by_posts_count,cited_by_tweeters_count,cited_by_videos_count,cited_by_wikipedia_count,cited_by_accounts_count,title,doi
0,702709,https://www.youtube.com/watch?v=GSqdhz--yR8,31.0,16.0,1.0,934.0,11.0,13.0,2.0,74.0,HLA genes in Macedonians and the sub-Saharan o...,10.1034/j.1399-0039.2001.057002118.x
1,403270,https://www.youtube.com/watch?v=2iqRF6mWj-A,9.0,2.0,1.0,698.0,97.0,58.0,2.0,173.0,Optimal low-density lipoprotein is 50 to 70 mg/dl,10.1016/j.jacc.2004.03.046
2,2394296,https://www.youtube.com/watch?v=k0HKvl5dYV0,115.0,5.0,37.0,1213.0,377.0,54.0,2.0,600.0,"Beyond Meatless, the Health Effects of Vegan D...",10.3390/nu6062131
3,632123,https://www.youtube.com/watch?v=GpfLHrtdcS4,62.0,83.0,1.0,1036.0,169.0,48.0,0.0,365.0,Twenty questions on atherosclerosis,0
4,411425,https://www.youtube.com/watch?v=2iqRF6mWj-A,23.0,8.0,8.0,896.0,333.0,27.0,1.0,410.0,Low-carbohydrate diets and all-cause and cause...,10.1059/0003-4819-153-5-201009070-00003


In [ ]:
df[:100].to_csv(path_or_buf = ".\\sample_dataset.csv", index=False)

In [75]:
url = "https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId=s9bHN0NUnrQ&key="

In [76]:
response = requests.get(url)
print(response.status_code)

200


In [77]:
json_data = json.loads(response.text)
json_data.keys()

dict_keys(['kind', 'etag', 'nextPageToken', 'pageInfo', 'items'])

In [101]:
json_data["items"][2]['snippet']['topLevelComment']['snippet']

{'authorDisplayName': 'Yeshi',
 'authorProfileImageUrl': 'https://yt3.ggpht.com/-F2F2ccFzzpE/AAAAAAAAAAI/AAAAAAAAAAA/9hR-4GhTFNg/s28-c-k-no-mo-rj-c0xffffff/photo.jpg',
 'authorChannelUrl': 'http://www.youtube.com/channel/UCAAvx670uu3QMCjZbC5tKSw',
 'authorChannelId': {'value': 'UCAAvx670uu3QMCjZbC5tKSw'},
 'videoId': 's9bHN0NUnrQ',
 'textDisplay': 'Le mieux qui peut arriver à la terre c&#39;est la disparition de l&#39;être humain, et au plus vite......',
 'textOriginal': "Le mieux qui peut arriver à la terre c'est la disparition de l'être humain, et au plus vite......",
 'canRate': True,
 'viewerRating': 'none',
 'likeCount': 0,
 'publishedAt': '2019-06-05T20:17:49.000Z',
 'updatedAt': '2019-06-05T20:17:49.000Z'}

In [ ]:
lc = 0
dc = 0
for j in len(json_data['items']):
    lc + = json_data['items'][j]["likeCount"]
    dc + = json_data['items'][j]["likeCount"]

In [ ]:
# add author information
def add_all_info(alt_id):
    # create the dict
    result = defaultdict(dict)
    try:
        # make the request
        response = requests.get("https://api.altmetric.com/v1/id/" + str(alt_id))

        # add the values to the dict
        if 'authors' in dict(json.loads(response.content)):
            result['author_count'] = len(dict(json.loads(response.content))['authors'])
            result['authors'] = ', '.join(dict(json.loads(response.content))['authors'])
        else:
            result['author_count'] = ''
            result['authors'] = ''

         # add the values to the dict
        if 'doi' in dict(json.loads(response.content)):
            result['doi'] = dict(json.loads(response.content))['doi']
        else:
            result['doi'] = ''

        # add the abstract
        if 'abstract' in dict(json.loads(response.content)):
            result['abstract'] = dict(json.loads(response.content))['abstract']
        else:
            result['abstract'] = ''

        # add the pubdate
        if 'published_on' in dict(json.loads(response.content)):
            result['pub_date'] = dict(json.loads(response.content))['published_on']
        else:
            result['pub_date'] = ''

        # add the twitter posts count as of 2018/19
        if 'cited_by_tweeters_count' in dict(json.loads(response.content)):
            result['twitter_count_y18'] = dict(json.loads(response.content))['cited_by_tweeters_count']
        else:
            result['twitter_count_y18'] = ''

        # add the title of the article
        if 'title' in dict(json.loads(response.content)):
            result['title'] =  dict(json.loads(response.content))['title']
        else:
            result['title'] = ''

        # return the result
        return result
    except:
        return result

In [ ]:
r = add_all_info(702709 )

In [6]:
r.keys()

NameError: name 'r' is not defined

In [115]:
rs_ = requests.get("https://api.altmetric.com/v1/id/" + str(2394296))

In [116]:
rs_js = json.loads(rs_.text)

In [117]:
rs_js

{'title': 'Beyond Meatless, the Health Effects of Vegan Diets: Findings from the Adventist Cohorts',
 'doi': '10.3390/nu6062131',
 'pmid': '24871675',
 'tq': ["#LifeAdviceIn5Words Eat Plants, You'll Live Longer #TasteEpcot #Vegan #healthy #cancer",
  'Vegetarian diets protect against cardiovascular diseases, some cancers and total mortality.',
  'Sources because I hate people who talk out of their asses',
  'Beyond Meatless, the Health Effects of Vegan Diets #Vegan #Health #NHS',
  'Beyond Meatless, the Health Effects of #Vegan Diets - 42% reduction in CVD mortality, 55% reduction of male IHD'],
 'altmetric_jid': '4f6fa6573cf058f610009228',
 'issns': ['2072-6643'],
 'journal': 'Nutrients',
 'cohorts': {'pub': 319, 'sci': 16, 'doc': 34, 'com': 7},
 'abstract': 'Vegetarians, those who avoid meat, and vegans, additionally avoiding dairy and eggs, represent 5% and 2%, respectively, of the US population. The aim of this review is to assess the effects of vegetarian diets, particularly stric

In [118]:
fl = open("temp.txt", 'w+')

In [119]:
json.dump(rs_js, fl)

In [ ]:
rs_js['cited_by_accounts_count']

In [ ]:
rs_js['cited_by_videos_count']

In [ ]:
response_statistics = requests.get(url_statistics)
print(response.status_code)

In [ ]:
json_statistics = json.loads(response_statistics.text)
json_statistics.keys()

In [ ]:
json_statistics['items']